In [13]:
#TODO s : 
#There must be no chain pipeline in the input original file.
# common stages are choose only by bing the exact same and not the one with higher data access
# no policy enforcement applied


In [14]:
!pip install ipynb
!pip install ruamel.yaml
#!pip install import-ipynb

In [15]:
from typing import List, Tuple
from ipynb.fs.full.kubeflowPipelineGenerator import * 
from difflib import SequenceMatcher
import uuid

In [16]:
def find_match_stages(pipeline_config: PipelineConfig) -> List[Tuple[List[List[str]], Pipeline, Pipeline]]:
    common_stages_list = []  # To store the result

    # Get all pipelines from the config
    pipelines = pipeline_config.pipelines

    # Iterate over every pair of pipelines
    for i in range(len(pipelines)):
        for j in range(i + 1, len(pipelines)):
            pipeline1 = pipelines[i]
            pipeline2 = pipelines[j]

            # Find the common stages in the same sequence using SequenceMatcher
            common_stages = find_contiguous_matches(pipeline1.flow, pipeline2.flow)

            # Only add non-empty common stages
            if common_stages:
                common_stages_list.append((common_stages, pipeline1, pipeline2))

    return common_stages_list

def find_contiguous_matches(flow1: List[str], flow2: List[str]) -> List[List[str]]:
    matcher = SequenceMatcher(None, flow1, flow2)
    common_stages = []

    for block in matcher.get_matching_blocks():
        if block.size > 0:  # Only consider blocks with matches
            start1 = block.a
            length = block.size

            # Collect the matched stages from flow1
            matched_stages = flow1[start1:start1 + length]

            # Only add matched stages as a separate list
            common_stages.append(matched_stages)

    return common_stages



In [17]:

# def filter_common_stages_by_length(threshold: int, common_stages: List[Tuple[List[List[str]], Pipeline, Pipeline]]) -> List[Tuple[List[List[str]], Pipeline, Pipeline]]:
#     """
#     Filters the common stages to only include those with sublist lengths greater than the threshold.
#     """
#     return [
#         (stages, pipeline1, pipeline2)
#         for stages, pipeline1, pipeline2 in common_stages
#         if any(len(sublist) > threshold for sublist in stages)  # Check for any sublist longer than the threshold
#     ]
def filter_common_stages_by_length(threshold: int, common_stages: List[Tuple[List[List[str]], Pipeline, Pipeline]]) -> List[Tuple[List[List[str]], Pipeline, Pipeline]]:
    """
    Filters the common stages to only include those sublists where their length is greater than the threshold.
    """
    filtered_stages = []
    
    for stages, pipeline1, pipeline2 in common_stages:
        # Keep only sublists with length greater than the threshold
        valid_stages = [sublist for sublist in stages if len(sublist) > threshold]
        
        # If there are any valid stages, add them to the filtered list
        if valid_stages:
            filtered_stages.append((valid_stages, pipeline1, pipeline2))

    return filtered_stages


In [18]:
#create new pipelines from common stages

# def create_new_pipeline(filtered_common_stages:List[Tuple[List[List[str]], Pipeline, Pipeline]]) -> Pipeline :
#     
#     new_pipeline_list =[] 
#     
#     for common_stage_names, pipeline1_obj, pipeline2_obj in filtered_common_stages:
#         for sublist in common_stage_names:
#             flow = [sublist]
#             consumers= [pipeline1_obj.consumers, pipeline2_obj.consumers]
#             datasource = Datasource(type="pipeline", metadata=pipeline1_obj.name+','+pipeline2_obj.name)
#             name='auto_generated_'+str(uuid.uuid4())
#             new_pipeline = Pipeline(name=name, flow=flow, datasource=datasource, consumers=consumers)
#             new_pipeline_list.append(new_pipeline)
#     return new_pipeline_list

In [19]:
def split_list_by_common_parts(pipeline_flow_list: List[str], common_sub_flow_list:List[List[str]])-> List[List[str]]:
    result = []
    i = 0

    for common_sublist in common_sub_flow_list:
        # Find the index of the common sublist in the main list
        start_idx = pipeline_flow_list.index(common_sublist[0], i)
        end_idx = start_idx + len(common_sublist)

        # Add the part before the common sublist, if it exists
        if start_idx > i:
            result.append(pipeline_flow_list[i:start_idx])

        # Add the common sublist itself
        result.append(pipeline_flow_list[start_idx:end_idx])

        # Update the index to continue after the common sublist
        i = end_idx

    # Add the remaining part of the list, if it exists
    if i < len(pipeline_flow_list):
        result.append(pipeline_flow_list[i:])

    return result
    
        

In [20]:
def generate_new_pipelines(filtered_common_stages: List[Tuple[List[List[str]], Pipeline, Pipeline]]) -> Tuple[List[Pipeline], List[PipelineChain]]:
    
    pipeline_list = []
    pipeline_chain_list = []

    
    for common_stage_lists, pipeline1_obj, pipeline2_obj in filtered_common_stages:
        pipeline_1_chain= PipelineChain(name=pipeline1_obj.name+'_'+ str(uuid.uuid4()),flow=[])
        pipeline_2_chain= PipelineChain(name=pipeline2_obj.name+'_'+ str(uuid.uuid4()),flow=[])
    
        pipeline1_sub_flows=split_list_by_common_parts(pipeline1_obj.flow,common_stage_lists)
        pipeline2_sub_flows=split_list_by_common_parts(pipeline2_obj.flow,common_stage_lists)
        
################################################################## pipeline_1_flow
        for flow in pipeline1_sub_flows:
            if flow not in common_stage_lists:
                 # Check if the flow exists in the pipeline_list
                pipeline_found = False
                for pipeline in pipeline_list:
                    if pipeline.flow == flow:
                        pipeline_1_chain.flow.append(pipeline.name)
                        pipeline_found = True
                        break
                
                # If pipeline with the flow wasn't found, create a new one and add it to the chain
                if not pipeline_found:
                    new_pipeline=Pipeline(
                        name=f"auto_gen_pipeline_{uuid.uuid4()}",
                        flow=flow,
                        datasource=Datasource(type="pipeline", metadata=pipeline1_obj.name),
                        consumers=pipeline1_obj.consumers
                    )
                    pipeline_list.append(new_pipeline)
                    
                    pipeline_1_chain.flow.append(new_pipeline.name)
            else:
                # Check if the flow exists in the pipeline_list
                pipeline_found = False
                for pipeline in pipeline_list:
                    if pipeline.flow == flow:
                        pipeline_1_chain.flow.append(pipeline.name)
                        pipeline_found = True
                        break
                
                # If pipeline with the flow wasn't found, create a new one and add it to the chain
                if not pipeline_found:
                    if isinstance(pipeline1_obj.consumers, str):
                        pipeline1_obj.consumers = [pipeline1_obj.consumers]
                    if isinstance(pipeline2_obj.consumers, str):
                        pipeline2_obj.consumers = [pipeline2_obj.consumers]   
                    new_pipeline = Pipeline(
                    name=f"auto_gen_pipeline_{uuid.uuid4()}",
                    flow=flow,
                    datasource=Datasource(type="pipeline", metadata=pipeline1_obj.name+','+pipeline2_obj.name),
                    consumers=pipeline1_obj.consumers.extend(pipeline2_obj.consumers)
                     )
                    pipeline_list.append(new_pipeline)
                    pipeline_1_chain.flow.append(new_pipeline.name)

        pipeline_chain_list.append(pipeline_1_chain)
        
########################################################### pipeline_2_flow
        for flow in pipeline2_sub_flows:
            if flow not in common_stage_lists:
                # Check if the flow exists in the pipeline_list
                pipeline_found = False
                for pipeline in pipeline_list:
                    if pipeline.flow == flow:
                        pipeline_2_chain.flow.append(pipeline.name)
                        pipeline_found = True
                        break
                
                # If pipeline with the flow wasn't found, create a new one and add it to the chain
                if not pipeline_found:
                    new_pipeline=Pipeline(
                        name=f"auto_gen_pipeline_{uuid.uuid4()}",
                        flow=flow,
                        datasource=Datasource(type="pipeline", metadata=pipeline2_obj.name),
                        consumers=pipeline2_obj.consumers
                    )
                    pipeline_list.append(new_pipeline)
                    pipeline_2_chain.flow.append(new_pipeline.name)
            else:
                # Check if the flow exists in the pipeline_list
                pipeline_found = False
                for pipeline in pipeline_list:
                    if pipeline.flow == flow:
                        pipeline_2_chain.flow.append(pipeline.name)
                        pipeline_found = True
                        break
                
                # If pipeline with the flow wasn't found, create a new one and add it to the chain
                if not pipeline_found:
                    if isinstance(pipeline1_obj.consumers, str):
                        pipeline1_obj.consumers = [pipeline1_obj.consumers]
                    if isinstance(pipeline2_obj.consumers, str):
                        pipeline2_obj.consumers = [pipeline2_obj.consumers]  
                        
                    new_pipeline = Pipeline(
                    name=f"auto_gen_pipeline_{uuid.uuid4()}",
                    flow=flow,
                    datasource=Datasource(type="pipeline", metadata=pipeline1_obj.name+','+pipeline2_obj.name),
                    consumers=pipeline1_obj.consumers.extend(pipeline2_obj.consumers)
                     )
                    pipeline_list.append(new_pipeline)
                    pipeline_2_chain.flow.append(new_pipeline.name)
                    
        pipeline_chain_list.append(pipeline_2_chain)   
        
    return pipeline_list, pipeline_chain_list                        

In [21]:
import os
import yaml

class NoAliasDumper(yaml.Dumper):
    def ignore_aliases(self, data):
        return True
    
def write_pipeline_config_to_yaml(config: PipelineConfig, path: str, filename: str):
    BASE_DATA_DIR = 'data/'
    
    # Ensure the directory exists
    full_dir_path = os.path.join(BASE_DATA_DIR, path)  # Path where the YAML will be saved
    os.makedirs(full_dir_path, exist_ok=True)  # Create the directory if it doesn't exist
    
    # Construct the full file path (ensure filename is valid)
    full_file_path = os.path.join(full_dir_path, filename)  # Complete file path
    
    # Write the YAML file
    with open(full_file_path, 'w') as file:
        yaml.dump(config.to_dict(), file, default_flow_style=False, allow_unicode=True,Dumper=NoAliasDumper,sort_keys=False)

In [22]:
# keep pipeline that has nothing in common
# generate a chain fo them to
# create the new pipeline config obj
def regenerate_pipeline_config_yml(pipeline_config:PipelineConfig,filtered_common_stages: List[Tuple[List[List[str]], Pipeline, Pipeline]],path:str, filename: str)-> Tuple[List[Pipeline], List[PipelineChain]]:
    
    new_pipeline_list, pipeline_chain_list = generate_new_pipelines(filtered_common_stages) 
    
    for pipeline in pipeline_config.pipelines:
        pipeline_exist = False
        for common_stage,pipeline_1,pipeline_2 in filtered_common_stages:
            if pipeline==pipeline_1 or pipeline_2 == pipeline :
                pipeline_exist = True
                break
        if not pipeline_exist:
            new_pipeline_list.append(pipeline)
            pipeline_chain_list.append(PipelineChain(name=pipeline.name , flow=pipeline.name))
    
    pipeline_config.pipelines = new_pipeline_list
    pipeline_config.pipeline_chains = pipeline_chain_list
    write_pipeline_config_to_yaml(pipeline_config,path,filename) 
    
    return new_pipeline_list, pipeline_chain_list

In [23]:

# Iterate over the config file to find common stages between pipelines
def parse_pipeline_config(config_path:str, new_config_path:str, new_config_name:str):
    
    pipeline_config =read_yaml_to_pipeline_config(config_path)
    
    common_stages_tuple= find_match_stages(pipeline_config)
    
    filtered_common_stages=filter_common_stages_by_length(0,common_stages_tuple)
    for common_stage_names, pipeline1_obj, pipeline2_obj in filtered_common_stages:
        print(f"Common stages between {pipeline1_obj.name} and {pipeline2_obj.name}: {common_stage_names}")
        
########### DEBUG #####################
    for common_stage_lists, pipeline1_obj, pipeline2_obj in filtered_common_stages:

        print(f"original_pipeline_1 :  name = {pipeline1_obj.name} , flow: {pipeline1_obj.flow}") 
        print(f"original_pipeline_2 :  name = {pipeline2_obj.name} , flow: {pipeline2_obj.flow}") 
        print(f"common_stages_pipelines : {common_stage_lists}") 

        test = split_list_by_common_parts(pipeline1_obj.flow,common_stage_lists)        
        test_2 = split_list_by_common_parts(pipeline2_obj.flow,common_stage_lists)  

        for list in test:
            print(f"split_list_by_common_parts_pipeline_1 : {list}")
        for list in test_2:
            print(f"split_list_by_common_parts_pipeline_2 : {list}")
        
######################################    
    
    new_pipeline_list, pipeline_chain_list = regenerate_pipeline_config_yml(pipeline_config,filtered_common_stages,new_config_path,new_config_name)
    
    for new_pipeline in new_pipeline_list:
        print(f"New auto-generated pipeline is {new_pipeline.name}: ,  stages are: {new_pipeline.flow}")

    for chain in pipeline_chain_list:
        print(f"New auto-generated chains are {chain.name}: pipelines are: {chain.flow}")        

    # create new pipeline config file
    

    return

In [24]:
def main():
    
    parse_pipeline_config(config_path='data/shipments_pipeline_config.yaml',new_config_path='new_pipeline', new_config_name='shipments_pipeline_config.yaml')

# Call the main function
if __name__ == "__main__":
    main()

Common stages between employee_data_pipeline and sales_data_pipeline: [['filter_data'], ['aggregate_data', 'compress_json_to_output']]
original_pipeline_1 :  name = employee_data_pipeline , flow: ['filter_data', 'anonymize_columns', 'aggregate_data', 'compress_json_to_output']
original_pipeline_2 :  name = sales_data_pipeline , flow: ['filter_data', 'aggregate_data', 'compress_json_to_output']
common_stages_pipelines : [['filter_data'], ['aggregate_data', 'compress_json_to_output']]
split_list_by_common_parts_pipeline_1 : ['filter_data']
split_list_by_common_parts_pipeline_1 : ['anonymize_columns']
split_list_by_common_parts_pipeline_1 : ['aggregate_data', 'compress_json_to_output']
split_list_by_common_parts_pipeline_2 : ['filter_data']
split_list_by_common_parts_pipeline_2 : ['aggregate_data', 'compress_json_to_output']
New auto-generated pipeline is auto_gen_pipeline_be167e83-d098-4252-9933-8f914c998daf: ,  stages are: ['filter_data']
New auto-generated pipeline is auto_gen_pipeline